In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
## load the dataset
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
## drop unnecessary columns

df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [4]:
## Encode categorial variable
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])


In [5]:
data=df

In [6]:
## one hot encode 'Geography

from sklearn.preprocessing import OneHotEncoder
oh_encder_geo=OneHotEncoder()
geo_encoded_value=oh_encder_geo.fit_transform(data[['Geography']]).toarray()

geo_encoded_value


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
geo_encoded_df=pd.DataFrame(data=geo_encoded_value,columns=oh_encder_geo.get_feature_names_out())

In [8]:
## combine one hot encoded columns with the encoded data
data = pd.concat([data.drop(columns="Geography"),geo_encoded_df],axis=1)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('oh_encoder_geo.pkl','wb') as file:
    pickle.dump(oh_encder_geo,file)

In [11]:
## Divide the data set into independent and dependent feature
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [12]:
## Scale this features

scaler =StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [13]:
## Saving the scaler file
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN Implementation

In [14]:
import tensorflow as ts
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [15]:
## Build the ann model
model=Sequential([
    ## First hidden layer connected with input layer
    ## (number of neurons , activation,number of inputs)
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## require for only first hidden layer, shape[1], blank means sinle dimensions
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')## Output layer
])

In [16]:
model.summary() ## prams = total bias + total bias 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
opt=ts.keras.optimizers.Adam(learning_rate=0.01)
## if uh direct pass adam in optimizer then it has fixed learning rate so use this instead
loss=ts.keras.losses.BinaryCrossentropy()


In [18]:
## Compile the model
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [19]:
## Setup the Tensorboard
log_dir='logs/fit'
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1) ## used to visualize your logs

In [20]:
## Set up early stopping
## if we are training our model for 100 epoch but our model is best fits in only 20 epochs then we need to stop it early
## be patience atleast 5 epoch then stop
## during backword propagation restore the best weights
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [21]:
### Training the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 6s 11ms/step - loss: 0.3940 - accuracy: 0.8393 - val_loss: 0.3451 - val_accuracy: 0.8595
Epoch 2/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3553 - accuracy: 0.8534 - val_loss: 0.3402 - val_accuracy: 0.8650
Epoch 3/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3481 - accuracy: 0.8584 - val_loss: 0.3528 - val_accuracy: 0.8620
Epoch 4/100
250/250 [==============================] - 3s 13ms/step - loss: 0.3455 - accuracy: 0.8583 - val_loss: 0.3414 - val_accuracy: 0.8620
Epoch 5/100
250/250 [==============================] - 4s 16ms/step - loss: 0.3418 - accuracy: 0.8599 - val_loss: 0.3418 - val_accuracy: 0.8585
Epoch 6/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3369 - accuracy: 0.8615 - val_loss: 0.3412 - val_accuracy: 0.8650
Epoch 7/100
250/250 [==============================] - 3s 12ms/step - loss: 0.3364 - accuracy: 0.8608 - val_loss: 0.3513 - val_accuracy

In [22]:
model.save('model.h5')

c:\Users\HP\OneDrive\Desktop\Ann Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
## Load tensorflow extension
%load_ext tensorboard

In [24]:
%tensorboard --logdir logs/fit ## savng logs

Reusing TensorBoard on port 6006 (pid 25820), started 20:31:24 ago. (Use '!kill 25820' to kill it.)